-----------------------
## 1) New way of scraping  11.4.2024
-----------------------

In [ ]:
from run import Runner
runner = Runner()

## Complete scraping + updating DB

In [ ]:
runner.run_complete_scraping(["prodej_byty", 
                              "prodej_domy",
                              "prodej_pozemky", 
                              "prodej_komercni",
                              "prodej_ostatni",
                              "pronajem_byty",
                              "pronajem_domy"])

In [ ]:
#runner.update_geodata()
#runner.input_all_estates_to_db()
#runner.input_all_prices_to_db()
#runner.scrape_prices_and_details(["count_prodej_byty", 
#                                "count_prodej_domy"])

## Test diagnostics

In [ ]:
from diagnostics.diagnostics import Diagnostics
import json 


diag = Diagnostics()

#result = diag.summary_new_estates()

#runner.mailing.send_email(subject=f'SCRAPING SREALITY SUMMARY for {result["latest_timestamp"]}',
#                        message_text=json.dumps(result))

diag.describe_database()

## Testing requests

In [ ]:
import requests
# Just testing count
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/count"
    result =  session.request(method="GET", url=url, headers=headers).json()
    print(result)
    url_filter = f"https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=1&page=1"
    result2 =  session.request(method="GET", url=url_filter).json()
    print(result2)

In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/95413580"
    result =  session.request(method="GET", url=url, headers=headers).json()
    display(result)


## Test Utils - timestamp and mail

In [ ]:
import json
from run import Runner
runner = Runner()

In [ ]:
timestamp, _ = runner.utils.generate_timestamp()
print(timestamp)
#data_manager._insert_new_estate(df_
# new, timestamp)

In [ ]:
runner.mailing.send_email("hallo", "toto je mesidz")


In [ ]:
runner.mailing.send_email_with_attachment("hallo", "toto je mesidz", 'scraping.log')


In [ ]:
#discounts_all = runner.diag.discounts_in_last_batch(filters=None)
#runner.mailing.send_email(subject=f'SREALITY - DISCOUNTS {discounts_all["Last Date"]}',
#                        message_text=json.dumps(discounts_all))

discounts_targeted = runner.diag.discounts_in_last_batch()

runner.mailing.send_email(subject=f'SREALITY - DISCOUNTS TARGETED {discounts_targeted["Last Date"]}',
                        message_text=json.dumps(discounts_targeted))

## Test Prepare Geo-JSONs

In [ ]:
source_folder= "c:\\Users\\jirka\\Documents\\MyProjects\\Sreality\\data_2024\\estate_details"
files = os.listdir(source_folder)
file_info = []

for file_name in files:
        file_path = os.path.join(source_folder, file_name)
        if os.path.isfile(file_path):
            size = os.path.getsize(file_path)
            file_info.append((file_name, size))

# Sort files by size
sorted_files = sorted(file_info, key=lambda x: x[1])

sorted_names = []
for file_name, size in sorted_files:
    sorted_names.append(file_name)
    print(f"{file_name}: {size} bytes")

In [ ]:
runner.geodata.enrich_jsons_with_geodata(list_of_jsons=None)

## Test table managment

In [ ]:
d = runner.data_manager.get_all_rows("price_history")
print(len(d))
d_codes = d["estate_detail"].unique()
print(len(d_codes))

In [ ]:
# price_history, estate_detail
runner.data_manager.clear_table("X")

In [ ]:
runner.data_manager.drop_table("x")

In [ ]:
runner.data_manager.create_table("x")

In [ ]:
upload = pd.DataFrame(data = [[42, "Prodej luxsního bejváku", "Byt 14+kk daleko od centra, ale v klidu.",
                               1, 6, 1, 111, "M&M reality", 2, 49.827491575, 18.1654443,
                               0, 0, 111083, 65, 14829, 12, 10, 4730, 112,
                               0, 0, 0, 0, 0, 2, 0, 79, "Družstevní", 0, 1, 1, 0,
                               3, 0, 870824, 1, 2,
                               ### Not mandatory:
                               "Cena je finální", 123,
                               "Dnes", "cihla", "Novostavba", "Družstevní", "2.patro",
                               42, 43, 2, "true"
                               ]],
                      columns=['code', 'description', 'meta_description', 
                               'category_main_cb', 'category_type_cb', 'category_sub_cb', 'broker_id', 'broker_company',
                            'furnished', 'latitude', 'longitude', 'locality', 'city', 'district',
                            'region', 'object_type', 'parking_lots', 'locality_street_id',
                            'locality_district_id', 'locality_ward_id', 'locality_region_id',
                            'locality_quarter_id', 'locality_municipality_id',
                            'locality_country_id', 'terrace', 'balcony', 'loggia', 'basin',
                            'cellar', 'building_type', 'object_kind', 'ownership', 'low_energy',
                            'easy_access', 'building_condition', 'garage', 'room_count_cb', 'energy_efficiency_rating_cb', 
                            ### Not mandatory:
                            'note_about_price', 'id_of_order',
                            'last_update', 'material', 'age_of_building', 'ownership_type', 'floor',
                            'usable_area', 'floor_area', 'energy_efficiency_rating', "no_barriers"
                            ])

runner.data_manager._insert_new_estate(upload, "2024-04-21 23:22:44")

#TODO: how about unit test for this?
#? should fail predictably as it breakes foreign key constraints i made:)
upload = pd.DataFrame(data = [["911", "4242", "12345000"]],
                      columns=["estate_id", "batch_id", "price"])

runner.data_manager.insert_new_price(upload)

## Test Scheduler

In [ ]:
import datetime as dt

from scheduler import Scheduler

score = 1

def foo():
    print(f"foo called at {dt.datetime.now()}")

schedule = Scheduler()

schedule.cyclic(dt.timedelta(seconds=6), foo)

In [ ]:
print(schedule)

In [ ]:
import time

while True:
    schedule.exec_jobs()
    time.sleep(1)

## Konec - Bonusy, jednorázovky

In [ ]:
describe = d["id"].describe()
describe

print(describe["min"])

print(d["id"].min())

In [ ]:

# One-time - all JSONs - rename key "code" into "estate_id", so it is unified

import os
import json

#folder_path = 'C://Users//jirka//Documents//MyProjects//Sreality//data_2024//estate_details'

json_files = [f for f in os.listdir(folder_path)]

for json_file in json_files:
    file_path = os.path.join(folder_path, json_file)
    
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    for item in data:
        if 'code' in item:
            item['estate_id'] = item.pop('code')
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)

In [ ]:
# One-time - all CSVs - rename column "code" into "estate_id", so it is unified

import os
import pandas as pd

#folder_path = 'C://Users//jirka//Documents//MyProjects//Sreality//data_2024//prices'

csv_files = [f for f in os.listdir(folder_path)]
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    df = pd.read_csv(file_path, sep=";")
    
    df.rename(columns={'code':'estate_id'}, inplace=True)
    
    df.to_csv(file_path, sep=";", encoding="utf-8", index=False)

In [ ]:
# trying to update json object, so i can get more info for estates

#? update jde přímo přes index v listu, nebo pomocí definice itemu skrz value klíče
#? nebo přes počet klíčů. Každopádně se updatují data, ale přebývající klíče a hodnoty zůstanou!
my_list = [{"estate_id": "123","timestamp": "20240512_1234", "data": "are ok"},
           {"estate_id": "456","timestamp": "20240512_1239"}
           ]

new_object = {"estate_id": "789","timestamp": "20240512_1111",
              "some_data": "ahojky"}

#my_list[1].update(new_object)
#if item["estate_id"] == "456":

for item in my_list:
    if len(item.keys())==2:
        item.update(new_object)

display(my_list)

In [ ]:
# hledání všech souborů estate_detail, kde se vyskytují některé moje duplicitky:
from tqdm import tqdm
import os
import json
import pandas as pd
from run import Runner
runner = Runner()

#duplicities = ["1125872972","168523084","1795913036","1846244684","1964733772","2293527884","2500932940",
#"2686154060","3306480972","337343820","3674961228","3893065036","622556492","747320652","756159820","935601484"]

folder_with_jsons_files= f"{runner.cf.project_path}/{runner.cf.data_folder}/{runner.cf.estate_details_folder}"
files = os.listdir(folder_with_jsons_files)

dupli_files = []
for file_name in tqdm(files):
    file_path = os.path.join(folder_with_jsons_files, file_name)

    with open(file_path, 'r') as file:
        data = json.load(file)
    for item in data:
        #? Rozhodně nepřidávat další loop přes 2000 duplicities, stačí už toto
        if item["estate_id"] in duplicities:
            dupli_files.append(file_name)
            #print(f"found {duplik} in file {file_name}")


In [ ]:
print(len(dupli_files))
res = list(set(dupli_files))
print(len(res))
display(sorted(res))

# '20240511_2353.json'
# '20240512_0004.json'
# '20240512_0008.json'
# '20240421_2322.json',
# '20240507_2250.json'